In [1]:
import os
import pandas as pd
import glob

In [2]:
os.chdir("/fs/scratch/PAS0439/Ming/virome_ecology_core_prkaryotes/results/08_inStrain_rug4941/instrain_profiles/")

In [3]:
coexist_genome = pd.read_csv("/fs/ess/PAS0439/MING/virome_ecology/results/R_project/genome_prophage_co_exist_rug4941.csv")
prophage_genome = pd.read_csv("/fs/ess/PAS0439/MING/virome_ecology/results/R_project/genome_prophage_rug4941.csv")

In [4]:
scaffold_to_genome = pd.read_csv("../genomes.stb", sep = "\t", names = ["scaffold", "genome"])

In [5]:
prophage_scaffold1 = pd.read_csv("../../04_prophage_rumen_mags/prophage_rug_hungate_scaffold_coordinates_for_propogate.tsv", sep = "\t")
prophage_scaffold2 = pd.read_csv("../../04_prophage_rumen_mags/old_without_rug4941_hungate/prophage_scaffold_coordinates_for_propogate.tsv", sep = "\t")
prophage_scaffold_full = pd.concat([prophage_scaffold1,prophage_scaffold2])
prophage_sf = set(prophage_scaffold_full.scaffold)

In [6]:
prophage_anno = pd.read_csv('/fs/ess/PAS0439/MING/virome_ecology/results/R_project/rumen_mags_hugate_rug_prophages_anno.csv')

*gene microdiversity for genome of species, which both prophage-carrying and non-carrying population co-exist.  
Samples from RUG4941 Nature Biotechnology paper*

In [7]:
df_list = []11
for f in glob.glob("*"):
    i = f.split(".profile")[0]
    path = f"{i}.profile/output/{i}.profile_gene_info.tsv"
    if os.path.exists(path):
        df = pd.read_csv(f"{i}.profile/output/{i}.profile_gene_info.tsv", sep = "\t").query('breadth > 0.99').query('coverage > 10')    
    else:
        path = f"{i}.profile/output/{i}.profile_gene_info.tsv.gz"
        df = pd.read_csv(f"{path}", sep = "\t").query('breadth > 0.99').query('coverage > 10')
        
    df1 = pd.merge(df, scaffold_to_genome, on = "scaffold")
    df1.loc[:,"id"] = i
    df1.genome = df1.genome.apply(lambda x: x.split(".fasta")[0])
    df2 = pd.merge(df1, coexist_genome, on = ["genome", "id"])
    
    # mark prophage gene 
    prophage_presented = set(df2.query('scaffold in @prophage_sf').scaffold)
    prophage_coordinate = prophage_scaffold_full.query('scaffold in @prophage_presented')[["scaffold", "start", "stop"]].drop_duplicates()
    for f in set(prophage_coordinate.scaffold):
        Start = list(prophage_coordinate.query('scaffold == @f').start)[0]
        Stop = list(prophage_coordinate.query('scaffold == @f').stop)[0]
        prophage_gene_index = df2.query('scaffold == @f').query('start >= @Start').query('end <= @Stop').index
        df2.loc[prophage_gene_index, "prophage_gene"] = "Yes"
        df2.prophage_gene = df2.prophage_gene.fillna("No")
    
    df_list.append(df2)
    
prophage_co_exist_microdiversity = pd.concat(df_list).reset_index(drop = True)[['scaffold', 'gene', 'gene_length', 'coverage', 'breadth',
       'breadth_minCov', 'nucl_diversity', 'start', 'end', 'direction',
       'dNdS_substitutions', 'pNpS_variants', "SNV_count", "SNV_S_count", "SNV_N_count", "SNS_count",  "SNS_S_count",  "SNS_N_count", "divergent_site_count", 'genome', 'id', 'Genera', 'Species',
       'prophage_presented', 'prevelance', 'prophage_gene']]   

*gene microdiversity for prophage carrying genome.  
Samples from RUG4941 Nature Biotechnology paper*

In [8]:
df_list = []
for f in glob.glob("*"):
    i = f.split(".profile")[0]
    path = f"{i}.profile/output/{i}.profile_gene_info.tsv"
    if os.path.exists(path):
        df = pd.read_csv(f"{i}.profile/output/{i}.profile_gene_info.tsv", sep = "\t").query('breadth > 0.99').query('coverage > 10')    
    else:
        path = f"{i}.profile/output/{i}.profile_gene_info.tsv.gz"
        df = pd.read_csv(f"{path}", sep = "\t").query('breadth > 0.99').query('coverage > 10')
        
    df1 = pd.merge(df, scaffold_to_genome, on = "scaffold")
    df1.loc[:,"id"] = i
    df1.genome = df1.genome.apply(lambda x: x.split(".fasta")[0])
    df2 = pd.merge(df1, prophage_genome, on = ["genome", "id"])
    
    # mark prophage gene 
    prophage_presented = set(df2.query('scaffold in @prophage_sf').scaffold)
    prophage_coordinate = prophage_scaffold_full.query('scaffold in @prophage_presented')[["scaffold", "start", "stop"]].drop_duplicates()
    for f in set(prophage_coordinate.scaffold):
        Start = list(prophage_coordinate.query('scaffold == @f').start)[0]
        Stop = list(prophage_coordinate.query('scaffold == @f').stop)[0]
        prophage_gene_index = df2.query('scaffold == @f').query('start >= @Start').query('end <= @Stop').index
        df2.loc[prophage_gene_index, "prophage_gene"] = "Yes"
        df2.prophage_gene = df2.prophage_gene.fillna("No")
    
    df_list.append(df2)

prophage_microdiversity = pd.concat(df_list).reset_index(drop = True)[['scaffold', 'gene', 'gene_length', 'coverage', 'breadth',
       'breadth_minCov', 'nucl_diversity', 'start', 'end', 'direction',
       'dNdS_substitutions', 'pNpS_variants', "SNV_count", "SNV_S_count", "SNV_N_count", "SNS_count",  "SNS_S_count",  "SNS_N_count", "divergent_site_count", 'genome', 'id', 'Genera', 'Species',
       'prophage_presented', 'prevelance', 'prophage_gene']]   

In [9]:
prophage_co_exist_microdiversity.to_csv("/fs/ess/PAS0439/MING/virome_ecology/results/R_project/prophage_co_exist_microdiversity_rug4941_tmp.csv", index = None)
prophage_microdiversity.to_csv("/fs/ess/PAS0439/MING/virome_ecology/results/R_project/prophage_microdiversity_rug4941.csv_tmp", index = None)